In [7]:
from langchain.llms import GooglePalm
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')

In [8]:
llm = GooglePalm(google_api_key= api_key, temperature=0.7)

C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
print(llm('how to make vada paav with time calculation'))

**Ingredients:**

* 1 cup urad dal
* 1/2 cup rice
* 1 teaspoon cumin seeds
* 1/2 teaspoon fennel seeds
* 1/2 teaspoon turmeric powder
* 1/2 teaspoon red chili powder
* 1/4 teaspoon asafoetida
* 1/4 teaspoon salt
* Oil for deep frying
* 2 green chilies, chopped
* 1/2 cup cilantro, chopped
* 2-3 cups water

**Instructions:**

1. Soak the urad dal and rice in water for 6-8 hours.
2. Drain the water and grind the urad dal and rice together to a smooth paste.
3. Add the cumin seeds, fennel seeds, turmeric powder, red chili powder, asafoetida, and salt to the batter.
4. Mix well to combine.
5. Heat oil in a deep fryer or wok to 375 degrees F (190 degrees C).
6. Drop spoonfuls of the batter into the hot oil and fry until golden brown, about 5 minutes per side.
7. Remove from the oil and drain on paper towels.
8. To make the vada pav, split a hot dog bun in half and stuff with a vada, some chopped green chilies, and cilantro.
9. Serve immediately.

**Total time:** 1 hour 30 minutes

**Yield:**

## Now we are connecting with our database from MySQL

In [10]:
from langchain.utilities import SQLDatabase

In [15]:
db_user ='root'
db_password ='root'
db_host ='localhost'
db_name ='atliq_tshirts'

In [16]:
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)

In [18]:
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [20]:
from langchain_experimental.sql import SQLDatabaseChain

In [25]:
db_chain = SQLDatabaseChain(llm = llm, database = db, verbose = True)

C:\ML Projects\LLM - Projects\SQL_LLM\sql_llm\Lib\site-packages\langchain_experimental\sql\base.py:75: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [29]:
q1 = db_chain.run('how many nike tshirts in small size')



> Entering new SQLDatabaseChain chain...
how many nike tshirts in small size
SQLQuery:SELECT count(*) FROM t_shirts WHERE brand = 'Nike' AND size = 'S'
SQLResult: [(3,)]
Answer:3
> Finished chain.


In [30]:
q1

'3'

In [31]:
q2 = db_chain.run('how much is the price of inventory for all small size tshirts?')



> Entering new SQLDatabaseChain chain...
how much is the price of inventory for all small size tshirts?
SQLQuery:SELECT SUM(t_shirts.price * t_shirts.stock_quantity) FROM t_shirts WHERE t_shirts.size = 'S'
SQLResult: [(Decimal('20689'),)]
Answer:20689
> Finished chain.


in the above example we could have like wrong answer i dont how did it manage to give correct answer bt we will giver it few more  defilcult so we will decide how we can improve it 

In [32]:
q3 = db_chain.run('if we have to sell all the levis tshorts today with discount applied how much our store will genrate')



> Entering new SQLDatabaseChain chain...
if we have to sell all the levis tshorts today with discount applied how much our store will genrate
SQLQuery:SELECT SUM(price * (1 - pct_discount)) FROM t_shirts JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'discounts.start_date' in 'where clause'")
[SQL: SELECT SUM(price * (1 - pct_discount)) FROM t_shirts JOIN discounts ON t_shirts.t_shirt_id = discounts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

see in this question is dificult to address like its complex query and many it is using its own genrel knowledge like usually 
wehen get this question there should be coulumns like start date of discount and end date of discount we cant give discount all days so problem was that in our database we did not have any columns like startdate and enddate theen it uses oown knowledege without looking into database so we have to solve this problem suing concept of "few shot learning" lets see how we will use it

In [41]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

q3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('17178.550000'),)]
Answer:17178.55
> Finished chain.


In [34]:
q4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('17482'),)]
Answer:17482
> Finished chain.


In [35]:
q5 = db_chain.run("How many white color Levi's t shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE color = 'White' AND brand = 'Levi'
SQLResult: [(70,), (17,), (15,)]
Answer:70
> Finished chain.


Once again above is the wrong answer. We need to use SUM(stock_quantity). Let's run the query explicitly. We will use the result of this query later on in the notebook

In [36]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('102'),)]
Answer:102
> Finished chain.


### Few shot learning

We will use few shot learning to fix issues we have seen so far

In [42]:
few_shots = [
#     {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
#      'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
#      'SQLResult': "Result of the SQL query",
#      'Answer' : qns1},
    
#     {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
#      'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
#      'SQLResult': "Result of the SQL query",
#      'Answer': qns2},
    
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': q3} ,
    
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : q4},
    
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : q5
     }
]

# Embedding

In [73]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [74]:
embeddings = HuggingFaceBgeEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [75]:
string = "How many white color Levi's shirt I have? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White',Result of the SQL query,q5"

In [76]:
emb = embedings.embed_query(string)

In [77]:
len(emb),emb[:5]

(384,
 [0.0745023638010025,
  0.09563492238521576,
  0.043689947575330734,
  0.0679817795753479,
  -0.06574059277772903])

In [78]:
to_vectorize = [''.join(i.values()) for i in few_shots]

In [79]:
to_vectorize

["If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n Result of the SQL query17178.55",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'Result of the SQL query17482",
 "How many white color Levi's shirt I have?SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'Result of the SQL query70"]

# VectorStore

we have to store this embeding somewhere so to store this embding we have vectorStore so we can store aur embeding into this

the job of VectorStore is to take input questions convert into embedings and find the similar looking few shots examples

In [80]:
from langchain.vectorstores import Chroma

In [85]:
# !pip install chromadb==0.4.15

In [86]:
vectorstore = Chroma.from_texts(to_vectorize,embeddings,few_shots)

ValueError: Expected EmbeddingFunction.__call__ to have the following signature: odict_keys(['self', 'input']), got odict_keys(['self', 'args', 'kwargs'])
Please see https://docs.trychroma.com/embeddings for details of the EmbeddingFunction interface.
Please note the recent change to the EmbeddingFunction interface: https://docs.trychroma.com/migration#migration-to-0416---november-7-2023 
